<center>
<a href="http://www.insa-toulouse.fr/" ><img src="http://www.math.univ-toulouse.fr/~besse/Wikistat/Images/logo-insa.jpg" style="float:left; max-width: 120px; display: inline" alt="INSA"/></a> 
<a href="http://wikistat.fr/" ><img src="http://www.math.univ-toulouse.fr/~besse/Wikistat/Images/wikistat.jpg" style="max-width: 250px; display: inline"  alt="Wikistat"/></a>
<a href="http://www.math.univ-toulouse.fr/" ><img src="http://www.math.univ-toulouse.fr/~besse/Wikistat/Images/logo_imt.jpg" style="float:right; max-width: 200px; display: inline" alt="IMT"/> </a>
</center>

# IA Framework.
## Lab 1  - Introduction to Pyspark.
#### Part 4 IntroductionTo  [SparkML](https://spark.apache.org/docs/latest/ml-guide.html) library (or *MLlib  DataFrame-based API*)   <a href="http://spark.apache.org/"><img src="http://spark.apache.org/images/spark-logo-trademark.png" style="max-width: 100px; display: inline" alt="Spark"/> </a> 

## Introduction

Since Spark 2.0 MlLib library which used only RDD is note developped any more. If it still can be used, no more functionnality will be added.

The main *machine learning* Spark library is now `SparkML`. `SparkML` use only *DataFrame*.

`SparkML` does not has as much functionality as `MlLib` today, but it will reach total compatibility in Spark 3.0

## Context

In [ ]:
sc

##  Elementary statistics 

Most of function used in *MLlib* notebook, does not exist on *SparkML* only correlation function and hypothesis testing are available so far.

### Vectors object

The *SparkML* library used `Vectors` object to manipulate array (similar to numpy).

In [ ]:
from numpy import array
np_vectors=array([1.0,0.0,2.0,4.0,0.0])
np_vectors

In [ ]:
from pyspark.ml.linalg import Vectors
denseVec2=Vectors.dense([1.0,0.0,2.0,4.0,0.0])
denseVec2

The code above build *DenseVector*.  *SparseVector* can be used for sparse object.

In [ ]:
sparseVec1 = Vectors.sparse(10, {0: 1.0, 2: 2.0, 6: 4.0})
sparseVec1

Another syntax

In [ ]:
sparseVec2 = Vectors.sparse(10, [0, 2, 6], [1.0, 2.0, 4.0])
sparseVec2

### Correlation

The `pyspark.ml.stat.Correlation` function enables to compute correlations ( *Pearson* et *Spearman*)  between columns of a *DataFrame*.

In [ ]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.stat import Correlation

data = [(Vectors.sparse(4, [(0, 1.0), (3, -2.0)]),),
        (Vectors.dense([4.0, 5.0, 0.0, 3.0]),),
        (Vectors.dense([6.0, 7.0, 0.0, 8.0]),),
        (Vectors.sparse(4, [(0, 9.0), (3, 1.0)]),)]


df = spark.createDataFrame(data, ["features"])

r1 = Correlation.corr(df, "features").head()
print("Pearson correlation matrix:\n" + str(r1[0]))

r2 = Correlation.corr(df, "features", "spearman").head()
print("Spearman correlation matrix:\n" + str(r2[0]))

### Summary Statistics

In [ ]:
r2

###  Hypothesis testing

In [ ]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.stat import ChiSquareTest

data = [(0.0, Vectors.dense(0.5, 10.0)),
        (0.0, Vectors.dense(1.5, 20.0)),
        (1.0, Vectors.dense(1.5, 30.0)),
        (0.0, Vectors.dense(3.5, 30.0)),
        (0.0, Vectors.dense(3.5, 40.0)),
        (1.0, Vectors.dense(3.5, 40.0))]
df = spark.createDataFrame(data, ["label", "features"])

r = ChiSquareTest.test(df, "features", "label").head()
print("pValues: " + str(r.pValues))
print("degreesOfFreedom: " + str(r.degreesOfFreedom))
print("statistics: " + str(r.statistics))

## ML Pipeline

*SparkML*, is based on the notion of **ML Pipeline**. 

A **ML Pipeline** allows to combine different steps of the ML process from data cleaning and processing to learning trough object call *pipeline* or *workflow*. 

### Estimator, Transformer, and Param

A **ML Pipeline** is build from three type of objects:


 * **Transformer**: is a function which will convert a *DataFrame* to another *DataFrame*. In most case, the *DataFrame* created is similar to the old one with a new columns . **Transformer** example: 
     * The `predict` function of a ML model is a **Transformer**. It will take a DataFrame as an input and return a new dataframe with the prediction column. 
     * A *StringIndexer* column will take a DataFrame with text inside and will return new dataframe with text converted as numerical value.
 
* **Estimator**: is an algorithm, apply on a *DataFrame* in order to build **Transformer**. E**Estimator** example:
    * A learning algorithm is an **Estimator**. Once it has been applied on a *DataFrame* it will build a **Transformer** able to perform prediction

* **Parameter**: An API shared by  **Transformer** and **Estimators** to specified parameters.

#### Example : Logistic Regression

In [ ]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.classification import LogisticRegression

training = spark.createDataFrame([
    (1.0, Vectors.dense([0.0, 1.1, 0.1])),
    (0.0, Vectors.dense([2.0, 1.0, -1.0])),
    (0.0, Vectors.dense([2.0, 1.3, 1.0])),
    (1.0, Vectors.dense([0.0, 1.2, -0.5]))], ["label", "features"])


test = spark.createDataFrame([
    (1.0, Vectors.dense([-1.0, 1.5, 1.3])),
    (0.0, Vectors.dense([3.0, 2.0, -0.1])),
    (1.0, Vectors.dense([0.0, 2.2, -1.5]))], ["label", "features"])

We create a  `LogisticRegression` **Estimator**: `lr`.


In [ ]:
lr = LogisticRegression(maxIter=10, regParam=0.01, featuresCol='features', labelCol='label', predictionCol='prediction', probabilityCol='probability')
lr.explainParams()

We apply the estimator on the train *DataFrame*. It produces a **Transformer**.

In [ ]:
model = lr.fit(training)

We then apply the **Transformer** on the test `DataFrame` .

In [ ]:
prediction = model.transform(test)

The result is a new *Dataframe* : *prediction* which is the test *Dataframe* with two new columns : prediction and probability.

In [ ]:
prediction

Those names has been specified within the definition of `lr` **Estimator** used to build `model` **Transformer**

results

In [ ]:
result = prediction.select("features", "label", "probability", "prediction").collect()
for row in result:
    print("features=%s, label=%s -> prob=%s, prediction=%s"
          % (row.features, row.label, row.probability, row.prediction))

### Pipeline

A **Pipeline** is an association of differeny **Transformers** and **Estimators** in order to specify a complete ML pipeline. 

To apply text classification we will apply these successive steps:

 * Convert text in list of word (Tokenizer)
 * Convert list of word into numerical value (Hashing TF)
 * Training
 * Prediction

All these step will be resume in a single object **Pipeline**.

**NB** Tokenizer and Hashing TF will be used here without explanation. They will be studied in details in the third lab of AIF module.

#### Example : Tokenize, Hash and logistic regression.

In [ ]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import HashingTF, Tokenizer

# DataFrame d'Apprentissage
training = spark.createDataFrame([
    (0, "a b c d e spark", 1.0),
    (1, "b d", 0.0),
    (2, "spark f g h", 1.0),
    (3, "hadoop mapreduce", 0.0)
], ["id", "text", "label"])


# DataFrame Test.
test = spark.createDataFrame([
    (4, "spark i j k"),
    (5, "l m n"),
    (6, "spark hadoop spark"),
    (7, "apache hadoop")
], ["id", "text"])


*Tokenizer* is an **Estimator** which allows to build a  tokenizer **Transformer** object.

In [ ]:
tokenizer = Tokenizer(inputCol="text", outputCol="words")

In [ ]:
df_tokenized = tokenizer.transform(training)
df_tokenized.select("words").take(4)

*Hashing TF* is an **Estimator** hashing TF **Transformer** object.

In [ ]:
hashingTF = HashingTF(inputCol="words", outputCol="features")

In [ ]:
df_hash= hashingTF.transform(df_tokenized)
df_hash.select("features").take(4)

The step aboves can be combine in a Pipeline along with the training step.

In [ ]:
tokenizer = Tokenizer(inputCol="text", outputCol="words")
hashingTF = HashingTF(inputCol=tokenizer.getOutputCol(), outputCol="features")
lr = LogisticRegression(maxIter=10, regParam=0.001)

In [ ]:
pipeline = Pipeline(stages=[tokenizer, hashingTF, lr])

Apply all steps on the training dataframe.

In [ ]:
model = pipeline.fit(training)

Prediction

In [ ]:
prediction = model.transform(test)
selected = prediction.select("id", "text", "probability", "prediction")
for row in selected.collect():
    rid, text, prob, prediction = row
    print("(%d, %s) --> prob=%s, prediction=%f" % (rid, text, str(prob), prediction))

**Exercise** As in the second notebook, use SparkML library to build a ML model in order to predict attack on the [KDD Cup 1999](http://kdd.ics.uci.edu/databases/kddcup99/kddcup99.html) dataset.


Try to use various **Transformer** within the pipeline you will build : https://spark.apache.org/docs/latest/ml-features.html (One-Hot encoder to user string variable, ACP to reduce number of features, Different methods to scale the data, etc..)